In [1]:
import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import pandas as pd
import xlsxwriter

In [2]:
def removeStopword(str):
    stop_words = set(stopwords.words('stopwordID.csv'))
    word_tokens = word_tokenize(str)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

#remove sentence which contains only one word
def removeSentence(str): 
    word = str.split()
    wordCount = len(word)
    if(wordCount<=1):
        str = ''
    
    return str

def cleaning(str):
    #removeMention
    str = re.sub("\s([@][\w_-]+)"," ",str)
    #remove URLs
    str = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', str)
    #remove punctuations
    str = re.sub(r'[^\w]|_',' ',str)
    #remove digit from string
    str = re.sub("\S*\d\S*", "", str).strip()
    #removeHashtag
    str = re.sub('#[^\s]+','',str)
    #remove RT/cc
    str = re.sub('RT|cc', '', str)
    #remove non-ascii
    str = unicodedata.normalize('NFKD', str).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #to lowercase
    str = str.lower()
    #Remove additional white spaces
    str = re.sub('[\s]+', ' ', str)
    
    return str
    
#slang word
def normalize_slang_word(str):
    text_list = str.split(' ')
    slang_words_raw = pd.read_csv('slang_word_list.csv', sep=',', header=None)
    slang_word_dict = {}
    
    for item in slang_words_raw.values:
        slang_word_dict[item[0]] = item[1]
        
        for index in range(len(text_list)):
            if text_list[index] in slang_word_dict.keys():
                text_list[index] = slang_word_dict[text_list[index]]
    
    return ' '.join(text_list) 

def remove_repeated_character(str):
    str = re.sub(r'(.)\1{2,}', r'\1', str)
    
    return str

def preprocessing(str):
    str = removeSentence(str)
    str = cleaning(str)   
    str = remove_repeated_character(str)
    str = normalize_slang_word(str)
    str = removeStopword(str)
    
    return str

In [3]:
fo = pd.read_csv('prabowo2.csv') #read excel file
tweet_p = fo['tweet_text']
workbook_p = xlsxwriter.Workbook('cleandataprabowo.xlsx')
worksheet_p = workbook_p.add_worksheet()

row = 0
col = 0

rowHeaders = ['text']
worksheet_p.write_row(row, col,  tuple(rowHeaders))
        
for p in tweet_p:
    tweet_p = preprocessing(p)
    rowValues = [tweet_p]
    row += 1
    worksheet_p.write_row(row, col, tuple(rowValues))
    
workbook_p.close()

In [4]:
fo = pd.read_csv('jokowi2.csv') #read excel file
tweet_j = fo['tweet_text']
workbook_j = xlsxwriter.Workbook('cleandatajokowi.xlsx')
worksheet_j = workbook_j.add_worksheet()

row = 0
col = 0

rowHeaders = ['text']
worksheet_j.write_row(row, col,  tuple(rowHeaders))
        
for j in tweet_j:
    tweet_j = preprocessing(j)
    rowValues = [tweet_j]
    row += 1
    worksheet_j.write_row(row, col, tuple(rowValues))
    
workbook_j.close()